In [177]:
import json
import requests
import pandas as pd # module to convert an address into latitude and longitude values
import folium
from geopy.geocoders import Nominatim #Module for converting the address into latitude and longitude value
from IPython.display import display
from bs4 import BeautifulSoup #Helps us to get the HTML structure of the page




In [178]:
Address = '75 Thorncliffe Park Dr, Toronto, Ontario'

geolocation = Nominatim(user_agent="foursquare_agent")
location = geolocation.geocode(Address)
latitude = location.latitude
longitude = location.longitude

query = "Food"
limit = 10
radius = 500
print(latitude, longitude)

43.7049593 -79.3419739


In [179]:
#Call to the api using the stored variables
url = "https://api.foursquare.com/v3/places/search?query={}&ll={},{}&radius={}".format(query,latitude,longitude, radius)

headers = {
    "Accept": "application/json",
    "Authorization": "fsq30W5Rz0P5HJTL4F1jGkXsg37CiGhYOejnEqQCPdKwfUI="
}

#Response from the call
response = requests.request("GET", url, headers=headers)

#Convert the data to JSON
json_data = json.loads(response.text)

#Get the result and store it in list
listt = json_data["results"]

#For every hash, find the key "name"
for restaurant in listt: 
    print(restaurant["name"])


mymap = folium.Map(location=[43.70, -79.34], zoom_start=15)
mymap.save('mymap.html')



Bamiyan Kabob
A&W Restaurant
Pasha Authentic Turkish Cuisine
Faley Restaurant
Afghan Cuisine
Swiss Chalet Rotisserie & Grill
Anatolian's Turkish Halal Restaurant
Popeye's Louisiana Kitchen
Hero Certified Burgers
Capital Sweet Smousa


In [180]:

torontodata = pd.read_csv('Neighbourhoods_data.csv')
#list(torontodata)
#torontodata.head(1)

#Display all the data from the coloumn LATITUDE
#torontodata.loc[:, ['AREA_ID']]

In [181]:
#Create a list to store postal codes and neighborhood names
postalCode=[]
neighborhood=[]

#Website to extract the data from
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#use to extract data 
soup = BeautifulSoup(source,'html.parser')

table = soup.find().tbody

#find all rows that contain the body tag
row = table.findAll('b')

#for each column in the the row, find the body text(postal code)
for column in row:
    #Add it to the postal code list
    postalCode.append(column.get_text())
    #print(column.get_text()) #Check what is printing out

#find all rows that contain the body tag
row = table.findAll('span')

#for each column in the the row, find the span text(neighborhood)
for column in row:
    #Add it to the neighborhood list
    neighborhood.append(column.get_text())
    #print(column.get_text())

#Add postal code and neighborhood to the dataframe
df = pd.DataFrame({'Postal Code':postalCode,'Neighborhood':neighborhood})




In [187]:
#Create a list to store postal codes and neighborhood names
postalCode=[]
neighborhood=[]
district = []

#Website to extract the data from
url = "https://www.zipcodesonline.com/2020/06/postal-code-of-toronto-in-2020.html"

#Read the html tables into a list
scrapper = pd.read_html(url,skiprows=2)

#Store the date into the list 
neighborhood = scrapper[0][1]
postalCode = scrapper[0][2]
district =  scrapper[0][3]

#Check if data is right
#print(neighborhood)
#print(postalCode)
#print(district)

#Create a data fram and store the scraped data
finalData = pd.DataFrame({'Postal Code':postalCode,'Neighborhood':neighborhood, 'District':district})
finalData.index += 1 #start the index from 1 and not zero
print(finalData)


    Postal Code       Neighborhood          District
1           M5H           Adelaide  Downtown Toronto
2           M1V    Agincourt North       Scarborough
3           M1S          Agincourt       Scarborough
4           M9V     Albion Gardens         Etobicoke
5           M8W          Alderwood         Etobicoke
..          ...                ...               ...
201         M4B  Woodbine Gardens,         East York
202         M4C   Woodbine Heights         East York
203         M2P    York Mills West        North York
204         M2L         York Mills        North York
205         M5R          Yorkville   Central Toronto

[205 rows x 3 columns]


In [192]:
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:
 
excelData = pd.read_csv('neighbourhood-profiles-2016.csv')
df = pd.DataFrame(excelData, )
print(excelData)

     Agincourt North Agincourt South-Malvern West       Alderwood  \
0                129                          128              20   
1     No Designation               No Designation  No Designation   
2             29,113                       23,757          12,054   
3             30,279                       21,988          11,904   
4             -3.90%                        8.00%           1.30%   
...              ...                          ...             ...   
2378           8,610                        8,775           3,130   
2379           5,445                        5,610           2,200   
2380           3,170                        3,145             925   
2381             135                          220              70   
2382           2,280                        2,170             245   

               Annex Banbury-Don Mills  Bathurst Manor Bay Street Corridor  \
0                 95                42              34                  76   
1     No Design